In [1]:
from matplotlib import pyplot as plt
from matplotlib import style
from tabulate import tabulate
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns

plt.style.use('dark_background')
sns.set_palette("Set1")

In [2]:
tr_f_df = pd.read_csv(filepath_or_buffer='./data/train_fea.csv')
cv_f_df = pd.read_csv(filepath_or_buffer='./data/cv_fea.csv')
te_f_df = pd.read_csv(filepath_or_buffer='./data/test_fea.csv')